In [67]:
import os
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Chemin vers la racine du projet
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # remonte d'un dossier
data_path = os.path.join(project_root, 'data')

print("Chemin vers data :", data_path)


Chemin vers data : /home/jupyter/Fatma_Aziz/movie_recommender/data


In [41]:
# Charger train, test et movies
train_df = pd.read_csv(os.path.join(data_path, 'train_ratings.csv'))
test_df = pd.read_csv(os.path.join(data_path, 'test_ratings.csv'))
df_movies = pd.read_csv(os.path.join(data_path, 'movies.csv'))

# Vérifier
print("Train shape :", train_df.shape)
print("Test shape  :", test_df.shape)
print("Movies shape:", df_movies.shape)


Train shape : (84271, 6)
Test shape  : (21068, 6)
Movies shape: (10329, 5)


In [30]:
import pickle

with open(os.path.join(data_path, 'user_encoder.pkl'), 'rb') as f:
    user_encoder = pickle.load(f)

with open(os.path.join(data_path, 'movie_encoder.pkl'), 'rb') as f:
    movie_encoder = pickle.load(f)


In [54]:
# filtrage collaboratif 
# user_idx et movie_idx sont déjà créés
user_movie_matrix = train_df.pivot(index='user_idx', columns='movie_idx', values='rating')

# Affichage rapide
print(user_movie_matrix.head())

movie_idx  0      1      2      3      4      5      6      7      8      \
user_idx                                                                   
0            NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1            5.0    NaN    2.0    NaN    3.0    NaN    NaN    NaN    NaN   
2            NaN    NaN    NaN    NaN    3.0    NaN    NaN    NaN    NaN   
3            NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4            4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

movie_idx  9      ...  10315  10316  10317  10318  10319  10320  10321  10322  \
user_idx          ...                                                           
0            NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1            NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2            NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3            NaN  ...    NaN    NaN    NaN    NaN    NaN    Na

In [68]:
# Remplacer les NaN par 0 pour calculer la similarité
user_movie_matrix_filled = user_movie_matrix.fillna(0)

# Similarité entre utilisateurs
user_similarity = cosine_similarity(user_movie_matrix_filled)

# Convertir en DataFrame pour plus de clarté
user_similarity_df = pd.DataFrame(user_similarity, 
                                  index=user_movie_matrix.index, 
                                  columns=user_movie_matrix.index)

In [70]:
user_movie_matrix_filled.head(100)

movie_idx,0,1,2,3,4,5,6,7,8,9,...,10315,10316,10317,10318,10319,10320,10321,10322,10323,10324
user_idx,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,4.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
# Moyenne des ratings par film
mean_ratings = user_movie_matrix.mean(axis=0)  # moyenne colonne par colonne (chaque film)

# Trier par ordre décroissant
mean_ratings_sorted = mean_ratings.sort_values(ascending=False)

# Afficher les 10 premiers
print(mean_ratings_sorted.head(10))


movie_idx
1632     5.0
10118    5.0
7362     5.0
9426     5.0
810      5.0
9440     5.0
7225     5.0
6895     5.0
6890     5.0
6783     5.0
dtype: float64


In [72]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Trainset

# Définir le format des données
reader = Reader(rating_scale=(train_df['rating'].min(), train_df['rating'].max()))

# Charger directement le trainset
train_data = Dataset.load_from_df(train_df[['user_idx', 'movie_idx', 'rating']], reader)
trainset = train_data.build_full_trainset()  # Utilise tout train_df

# Charger directement le testset (test_df)
testset = list(zip(test_df['user_idx'], test_df['movie_idx'], test_df['rating']))

# Créer et entraîner le modèle
algo = SVD()
algo.fit(trainset)

# Prédictions sur test set
predictions = algo.test(testset)

# RMSE
rmse = accuracy.rmse(predictions)
print("RMSE sur le test set:", rmse)


RMSE: 0.8701
RMSE sur le test set: 0.8701062320893281


In [73]:
# Afficher les 10 premières prédictions et les vraies valeurs
print("user_idx | movie_idx | rating réel | rating prédit")
for pred in predictions[:10]:
    print(f"{pred.uid}        {pred.iid}         {pred.r_ui:.2f}        {pred.est:.2f}")


user_idx | movie_idx | rating réel | rating prédit
667        9012         3.00        2.80
158        4314         1.00        2.37
109        523         5.00        3.90
388        315         3.00        3.30
309        1889         3.00        2.84
457        3336         3.00        3.52
527        745         4.50        4.28
660        6         4.00        3.33
65        2388         4.00        3.67
6        2143         5.00        3.11


In [75]:
import matplotlib.pyplot as plt
import seaborn as sns
# Matrice prédite
n_users = train_df['user_idx'].nunique()
n_movies = train_df['movie_idx'].nunique()

pred_matrix = np.zeros((n_users, n_movies))

for uid in range(n_users):
    for iid in range(n_movies):
        pred_matrix[uid, iid] = algo.predict(uid, iid).est
        
pred_df = pd.DataFrame(pred_matrix, 
                       index=[f"User {uid}" for uid in range(n_users)],
                       columns=df_movies['title'].values[:n_movies])  # limiter au nombre de films entraînés

# Afficher les 10 premières lignes et colonnes
print(pred_df.iloc[:10, :10])



        Li'l Quinquin (    )  Pablo (2012)  The Big Broadcast of 1936 (1935)  \
User 0              3.792993      2.817311                          3.152700   
User 1              4.048791      3.619498                          3.109928   
User 2              3.885258      3.444549                          3.388643   
User 3              3.983766      3.583998                          3.618543   
User 4              3.878090      3.065206                          3.053539   
User 5              4.093715      3.719500                          3.409392   
User 6              4.215289      2.902428                          3.308741   
User 7              4.242909      3.514107                          3.748600   
User 8              2.957986      2.677535                          2.738809   
User 9              3.714167      3.405569                          3.206438   

        Round Trip to Heaven (1992)  The 50 Year Argument (2014)  \
User 0                     3.214664                

In [80]:
# recommandation for users 
def recommand_movie_to_user(user_id):
    # Tous les movie_idx vus par l'utilisateur
    movies_watched = train_df[train_df['user_idx']==user_id]['movie_idx'].tolist()

    # Tous les movie_idx connus par le modèle
    all_movie_idx = list(movie_encoder.transform(movie_encoder.classes_))

    pred_for_user = []

    for movie_idx in all_movie_idx:
        if movie_idx not in movies_watched:
            pred_rating = algo.predict(user_id, movie_idx).est
            pred_for_user.append((movie_idx, pred_rating))

    # Trier par note prédite décroissante
    top_recommendations = sorted(pred_for_user, key=lambda x: x[1], reverse=True)[:5]

    # Afficher
    print("Top 5 recommandations pour user", user_id)
    for movie_idx, rating in top_recommendations:
        movie_id = movie_encoder.inverse_transform([movie_idx])[0]
        title = df_movies[df_movies['movieId']==movie_id]['title'].values[0]
        print(f"{title} - predicted rating: {rating:.2f}")


In [81]:
recommand_movie_to_user(0)

Top 5 recommandations pour user 0
Under Siege 2: Dark Territory (1995) - predicted rating: 4.02
I Still Know What You Did Last Summer (1998) - predicted rating: 3.95
Matrix, The (1999) - predicted rating: 3.90
Star Trek IV: The Voyage Home (1986) - predicted rating: 3.83
Ace Ventura: When Nature Calls (1995) - predicted rating: 3.79


In [82]:
recommand_movie_to_user(50)

Top 5 recommandations pour user 50
Ocean's Eleven (2001) - predicted rating: 4.68
Lord of the Rings: The Two Towers, The (2002) - predicted rating: 4.52
Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000) - predicted rating: 4.43
I Still Know What You Did Last Summer (1998) - predicted rating: 4.37
Ace Ventura: When Nature Calls (1995) - predicted rating: 4.33


In [90]:
from sklearn.metrics.pairwise import cosine_similarity

def users_close (user_idx):
    # Similarité utilisateur × utilisateur
    user_similarity = cosine_similarity(pred_matrix)
    user_similarity_df = pd.DataFrame(user_similarity, index=all_user_ids, columns=all_user_ids)
    closest_users = user_similarity_df[user_idx].sort_values(ascending=False)[1:15]  # top 5
    print(closest_users)


In [91]:
users_close(0)


171    0.999418
286    0.999410
156    0.999408
476    0.999350
386    0.999340
343    0.999336
418    0.999324
597    0.999311
372    0.999309
40     0.999304
640    0.999296
333    0.999290
612    0.999281
176    0.999280
Name: 0, dtype: float64


In [92]:
users_close(50)

227    0.999531
387    0.999523
409    0.999515
257    0.999505
652    0.999496
212    0.999494
407    0.999486
145    0.999484
194    0.999480
146    0.999478
125    0.999470
591    0.999464
162    0.999463
100    0.999463
Name: 50, dtype: float64
